In [9]:
# On commence par importer les librairies nécessaires pour l'analyse des données.

import duckdb
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
import os

# On importe maintenant les données SISE-eaux, pour l'instant hebergé en "local" dans le GitHub en attendant le travail des data eng

# res: résultats des prélèvements, commune: liste des communes et réseaux UDI, prelev: détails administratifs des prélèvements

res = pd.read_csv(
    r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_RESULT_2024.txt"
)

commune = pd.read_csv(
    r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_COM_UDI_2024.txt"
)

prelev = pd.read_csv(
    r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data_temporaire\DIS_PLV_2024.txt"
)

ModuleNotFoundError: No module named 'geopandas'

In [2]:
# On affiche le haut du dataframe res

res.head()

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl
0,1,100138503,ALTMICR,1370.0,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,NaN,N,L,<10,µg/L,133,NaN,<=200 µg/L,0.00,7429-90-5,100147233.0
1,1,100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),NaN,O,L,Aspect normal,SANS OBJET,X,NaN,NaN,0.00,NaN,100147232.0
2,1,100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),NaN,O,L,Aspect normal,SANS OBJET,X,NaN,NaN,0.00,NaN,100147233.0
3,1,100138503,CL2LIB,1398.0,CHLORE LIBRE,Chlore libre,NaN,N,T,"0,30",mg(Cl2)/L,165,NaN,NaN,0.30,NaN,NaN
4,1,100138503,CL2TOT,1399.0,CHLORE TOTAL,Chlore total,NaN,N,T,"0,34",mg(Cl2)/L,165,NaN,NaN,0.34,NaN,NaN


In [3]:
res.columns

Index(['cddept', 'referenceprel', 'cdparametresiseeaux', 'cdparametre',
       'libmajparametre', 'libminparametre', 'libwebparametre', 'qualitparam',
       'insituana', 'rqana', 'cdunitereferencesiseeaux', 'cdunitereference',
       'limitequal', 'refqual', 'valtraduite', 'casparam', 'referenceanl'],
      dtype='object')

Les colonnes qui vont donc nous intéresser, au moins dans un premier temps, sont: referenceprel, 

In [5]:
prelev.head()

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim
0,1,1000003,1007,AMBRONAY,NaN,NaN,NaN,100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,N,C
1,1,1000003,1007,AMBRONAY,NaN,NaN,NaN,100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C
2,1,1000003,1007,AMBRONAY,1001304.0,TTP (CLG) AMBRONAY,100 %,100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C
3,1,1000003,1007,AMBRONAY,NaN,NaN,NaN,100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C
4,1,1000003,1007,AMBRONAY,NaN,NaN,NaN,100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C


In [4]:
commune.head()

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07


In [ ]:
# Lien avec une autre BDD pour obtenir les coordonnées géographiques des UDI

udi = pd.read_csv(
    r"C:\Users\emma.david\Desktop\data for good\eau test\draft-eau-tests\database\data eaurob\CAP_PLV_202411.txt"
)